In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../project"))

# Any results you write to the current directory are saved as output.

['.ipynb_checkpoints', 'Dataset', 'dsdsdsd', 'kernel4490903bdc.ipynb', 'Project.ipynb', 'project.py', 'SongRecommendation.py', 'SongRecommendationCollab.py', 'Untitled.ipynb']


In [3]:
jobsUSData = pd.read_csv('Dataset For Job Recommendation System/jobsUsdata.csv')
jobsUSData.head(4)

,Unnamed: 0,JobID,WindowID,Title,Description,Requirements,City,State,Country,Zip5,StartDate,EndDate
0,0,1,1,Security Engineer/Technical Lead,Security Engineer/Technical Lead<p>Security Cl...,<p>SKILL SET</p>\r<p>&nbsp;</p>\r<p>Network Se...,Washington,DC,US,20531.0,2012-03-07 13:17:01.643,2012-04-06 23:59:59
1,1,4,1,SAP Business Analyst / WM,SAP Business Analyst / WM<strong>NO Corp. to C...,<p><b>WHAT YOU NEED: </b></p>\r<p>Four year co...,Charlotte,NC,US,28217.0,2012-03-21 02:03:44.137,2012-04-20 23:59:59
2,2,7,1,P/T HUMAN RESOURCES ASSISTANT,P/T HUMAN RESOURCES ASSISTANT<b> <b> P/T HU...,Please refer to the Job Description to view th...,Winter Park,FL,US,32792.0,2012-03-02 16:36:55.447,2012-04-01 23:59:59
3,3,8,1,Route Delivery Drivers,Route Delivery DriversCITY BEVERAGES Come to w...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:10.077,2012-04-02 23:59:59


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [ ]:
jobsUSbase = jobsUSData.iloc[0:20000]

In [ ]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1,2),min_df=0,stop_words='english')
tfidf_matrix = tf.fit_transform(jobsUSbase['Description'])
tfidf_matrix.shape

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape
cosine_sim[0]
jobsUSbase.reset_index()

In [ ]:
titles  = jobsUSbase['Title']
indices = pd.Series(jobsUSbase.index, index=jobsUSbase['Title'])


In [ ]:
def get_recommnedation(title,cnt):
    similar_job_indices = []
    #def get_recommendation(title, no_of_jobs):
    i = indices[title]
    simulated_scores = enumerate(cosine_sim[i])
    simulated_scores = sorted(simulated_scores, key = lambda x: x[1],reverse = True)
    #print(simulated_scores)
    count = 0
    for i in simulated_scores:
        count = count+1
        similar_job_indices.append(i[0])
        if count == cnt:
            break
    print(similar_job_indices)
    for i in similar_job_indices:
        print(titles[i])  
    

In [ ]:
get_recommnedation('SAP Business Analyst / WM',10)

In [ ]:
get_recommnedation('Route Delivery Drivers',10)

In [ ]:
userData = pd.read_csv('../Project/Dataset For Job Recommendation System/users.tsv',delimiter = '\t')
userData.head()

In [ ]:
user_based_approach_US = userData.loc[userData['Country']=='US']
user_based_approach_US.head()

In [ ]:
user_based_approach_US = user_based_approach_US.iloc[0:15000]
user_based_approach_US

In [ ]:
appsData = pd.read_csv('../Project/Dataset For Job Recommendation System/apps.tsv',delimiter = '\t')
appsData.head()

In [ ]:
user_based_approach_US['Major'] = user_based_approach_US['Major'].fillna('')
user_based_approach_US['DegreeType'] = user_based_approach_US['DegreeType'].fillna('')
user_based_approach_US['TotalYearsExperience'] = str(user_based_approach_US['TotalYearsExperience'].fillna(''))
user_based_approach_US['Details'] = user_based_approach_US['Major'] + user_based_approach_US['DegreeType'] + user_based_approach_US['TotalYearsExperience']

In [ ]:
user_based_approach_US.head()

In [ ]:
tf = TfidfVectorizer(analyzer = 'word', ngram_range = (1,2), min_df =0, stop_words = 'english')
tfidf_matrix = tf.fit_transform(user_based_approach_US['Details'])

In [ ]:
tfidf_matrix.shape

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim[0]

In [ ]:
user_based_approach_US = user_based_approach_US.reset_index()
userid = user_based_approach_US['UserID']
indices = pd.Series(user_based_approach_US.index, index=user_based_approach_US['UserID'])

In [ ]:
def get_recommendations_userwise(userid,cnt):
    user_indices = []
    idx = indices[userid]
    sim_scores = enumerate(cosine_sim[idx])
    sim_scores = sorted(sim_scores, key = lambda x:x[1], reverse = True)
    count = 0
    for i in sim_scores:
        count = count+1
        user_indices.append(i[0])
        if count == cnt:
            break
    return user_indices

In [ ]:
print("---- Top N similar users with userId: 123 ---")
get_recommendations_userwise(123,10)

In [ ]:
def get_job_id(userid_list):
    jobs_userwise = appsData['UserID'].isin(userid_list)
    df1 = pd.DataFrame(data = appsData[jobs_userwise], columns=['JobID'])
    joblist = df1['JobID'].tolist()
    Job_list = jobsUSData['JobID'].isin(joblist)
    df_temp = pd.DataFrame(data = jobsUSData[Job_list], columns = ['JobID','Title','Description','State','City'])
    return df_temp
    

In [ ]:
get_recommendations_userwise(123,10)

In [ ]:
get_job_id(get_recommendations_userwise(123,10))

In [ ]:
get_recommendations_userwise(80,5)

In [ ]:
get_job_id(get_recommendations_userwise(80,5))

In [ ]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(jobsUSbase['Description'])

count_matrix.shape

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

cosine_sim2.shape


In [ ]:
def get_recommnedation(title,cnt):
    similar_job_indices = []
    #def get_recommendation(title, no_of_jobs):
    i = indices[title]
    simulated_scores = enumerate(cosine_sim2[i])
    simulated_scores = sorted(simulated_scores, key = lambda x: x[1],reverse = True)
    #print(simulated_scores)
    count = 0
    for i in simulated_scores:
        count = count+1
        similar_job_indices.append(i[0])
        if count == cnt:
            break
    print(similar_job_indices)
    for i in similar_job_indices:
        print(titles[i])


In [ ]:
get_recommnedation('SAP Business Analyst / WM',10)

In [ ]:
get_recommnedation('Route Delivery Drivers',10)